In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import os

In [ ]:
block1 = tf.keras.Sequential([tf.keras.layers.Conv2D(512, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.Conv2D(512, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,10,10,512]

block2 = tf.keras.Sequential([tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,5,5,512]
        
block3 = tf.keras.Sequential([tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,3,3,512]
        
block4 = tf.keras.Sequential([tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(512, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,1,1,512]

In [ ]:
mobilenet_v2 = MobileNetV2(input_shape=(300,300,3), weights="imagenet", include_top=False)

In [ ]:
tf.keras.utils.plot_model(mobilenet_v2, show_shapes=True, expand_nested=True)

# Build feature extractor

In [ ]:
# feature extractor
fmap38x38 = mobilenet_v2.get_layer(mobilenet_v2.layers[57].name).output #[BN,38,38,128]
fmap19x19 = mobilenet_v2.get_layer(mobilenet_v2.layers[119].name).output #[BN,19,19,576]
feature_extractor = tf.keras.Model(inputs=[mobilenet_v2.input], outputs=[fmap38x38, fmap19x19])

# Build extra convolutional layers

In [ ]:
input_layer = tf.keras.Input(shape=(300,300,3))
y1, y2 = feature_extractor(input_layer)
y3 = block1(y2)
y4 = block2(y3)
y5 = block3(y4)
y6 = block3(y5)

In [ ]:
print(y1.shape)
print(y2.shape)
print(y3.shape)
print(y4.shape)
print(y5.shape)
print(y6.shape)

In [ ]:
num_classes = 3

In [ ]:
#conf header
confs = []
conf = layers.Conv2D(4 * num_classes, kernel_size=3, padding='same')(y1)
conf = tf.reshape(conf, [-1, 38*38*4,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y2)
conf = tf.reshape(conf, [-1, 19*19*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y3)
conf = tf.reshape(conf, [-1, 10*10*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y4)
conf = tf.reshape(conf, [-1, 5*5*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y5)
conf = tf.reshape(conf, [-1, 3*3*4,num_classes])
confs.append(conf)
conf = layers.Conv2D(4 * num_classes, kernel_size=1)(y6)
conf = tf.reshape(conf, [-1, 1*1*4,num_classes])
confs.append(conf)

#location header
locs = []
loc = layers.Conv2D(4 * 4, kernel_size=3, padding='same')(y1)
loc = tf.reshape(loc, [-1, 38*38*4,4])
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y2)
loc = tf.reshape(loc, [-1, 19*19*6,4])
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y3)
loc = tf.reshape(loc, [-1, 10*10*6,4])
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y4)
loc = tf.reshape(loc, [-1, 5*5*6,4])
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y5)
loc = tf.reshape(loc, [-1, 3*3*4,4])
locs.append(loc)
loc = layers.Conv2D(4 * 4, kernel_size=1)(y6)
loc = tf.reshape(loc, [-1, 1*1*4,4])
locs.append(loc)

confs = tf.concat(confs, axis=-2)
locs = tf.concat(locs, axis=-2)

In [ ]:
mobilenet_ssd = tf.keras.Model(inputs=[input_layer], outputs=[confs,locs])

In [ ]:
tf.keras.utils.plot_model(mobilenet_ssd, show_shapes=True, expand_nested=False)

In [ ]:
mobilenet_ssd.summary()

In [ ]:
mobilenet_ssd.save("ssd300-mobilenet.h5")